In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats,
                                    leaguedashplayerptshot)
import time

In [2]:
current_year="2023-24"

In [3]:
pbp2023=pd.read_csv(filepath_or_buffer="Data/"+current_year+"_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [4]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season=current_year,season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season=current_year,
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [5]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [6]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
230,Jaylin Williams,1.146667
358,Moritz Wagner,1.107692
79,Cody Zeller,0.881633
45,Brandin Podziemski,0.827807
202,Jalen Brunson,0.748315
456,Tim Hardaway Jr.,0.615273
292,Kevin Love,0.589831
181,Isaiah Joe,0.551196
342,Marvin Bagley III,0.517127
307,Kyle Lowry,0.454608


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
71,Chris Duarte,1.954967
28,Ausar Thompson,1.851064
207,Jalen McDaniels,1.839130
413,Robert Covington,1.828877
265,Josh Okogie,1.791855
101,Dante Exum,1.698113
19,Andre Jackson Jr.,1.674419
402,Paul Reed,1.665306
420,Russell Westbrook,1.631461
211,Jalen Suggs,1.619331


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
162,Gary Payton II,6.679769
413,Robert Covington,6.275936
344,Matisse Thybulle,5.598238
11,Alex Caruso,4.784615
402,Paul Reed,4.775510
192,Jacob Gilyard,4.669091
146,Dyson Daniels,4.609091
59,Cam Reddish,4.116456
250,Jonathan Isaac,4.110448
18,Andre Drummond,4.036364


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [9]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
293,Kevon Looney,16.603448
109,Day'Ron Sharpe,15.798726
242,Jock Landale,15.600000
402,Paul Reed,14.693878
179,Isaiah Hartenstein,14.145882
144,Dwight Powell,14.019718
24,Anthony Davis,14.000000
79,Cody Zeller,13.628571
97,Daniel Gafford,13.410359
133,Domantas Sabonis,13.333333


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [10]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)

In [11]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Hustle Ranks.csv")

In [12]:
hustle_50percent_gp_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
161,Gary Harris,0.317365,0.017964,0.107784,0.458084,0.092814,0.223054,1.217066
299,Klay Thompson,0.317365,0.482036,0.149701,0.095808,0.182635,0.074850,1.302395
426,Sam Merrill,0.958084,0.101796,0.002994,0.185629,0.025449,0.034431,1.308383
138,Doug McDermott,0.317365,0.200599,0.290419,0.041916,0.446108,0.034431,1.330838
155,Fred VanVleet,0.317365,0.194611,0.059880,0.517964,0.170659,0.071856,1.332335
204,Jalen Green,0.317365,0.377246,0.182635,0.317365,0.104790,0.080838,1.380240
430,Scoot Henderson,0.317365,0.056886,0.104790,0.173653,0.221557,0.559880,1.434132
329,Malcolm Brogdon,0.317365,0.149701,0.263473,0.323353,0.125749,0.257485,1.437126
163,Gary Trent Jr.,0.317365,0.011976,0.095808,0.398204,0.122754,0.514970,1.461078
2,AJ Griffin,0.317365,0.239521,0.146707,0.077844,0.287425,0.428144,1.497006


In [13]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
398,Patrick Beverley,0.760479,0.137725,0.622754,0.727545,0.068862,0.826347,3.143713
420,Russell Westbrook,0.889222,0.251497,0.032934,0.823353,0.215569,0.940120,3.152695


# The "Got that Dawg in Him" Award (presented by Air Bud)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [14]:
hustle_50percent_gp_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
402,Paul Reed,0.317365,0.958084,0.889222,0.979042,0.868263,0.937126,4.949102
139,Draymond Green,0.988024,0.736527,0.898204,0.655689,0.976048,0.643713,4.898204
198,Jae'Sean Tate,0.931138,0.691617,0.826347,0.748503,0.712575,0.889222,4.799401
358,Moritz Wagner,0.994012,0.847305,0.805389,0.718563,0.982036,0.443114,4.790419
230,Jaylin Williams,0.997006,0.856287,0.856287,0.676647,0.937126,0.458084,4.781437
175,Herbert Jones,0.946108,0.571856,0.958084,0.922156,0.395210,0.934132,4.727545
355,Miles McBride,1.000000,0.182635,0.979042,0.991018,0.547904,0.832335,4.532934
179,Isaiah Hartenstein,0.317365,0.913174,0.574850,0.931138,0.880240,0.859281,4.476048
446,Tari Eason,0.317365,0.808383,0.949102,0.781437,0.814371,0.796407,4.467066
180,Isaiah Jackson,0.317365,0.982036,0.904192,0.500000,0.847305,0.901198,4.452096


# The Trickshot Grenadier Award (presented by Dude Perfect)

Highest sum of percentile ranks in FGA, FGA frequency & eFG% on shots with 4 seconds or less on the shotclock (minimum 50th percentile in FGA) (credit to BehavioralSink & Bylanta for the idea)

In [16]:
late_shotclock_shots=leaguedashplayerptshot.LeagueDashPlayerPtShot(shot_clock_range_nullable="4-0 Very Late").league_dash_ptshots.get_data_frame().loc[:, 'PLAYER_ID':'EFG_PCT']
late_shotclock_shots["FGA_percentile"]=late_shotclock_shots.FGA.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile=late_shotclock_shots.query("FGA_percentile>=0.5").copy()
late_shotclock_shots_top_50_percentile["FGA_percentile_among_top_50"]=late_shotclock_shots_top_50_percentile.FGA.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["FGA_FREQ_percentile"]=late_shotclock_shots_top_50_percentile.FGA_FREQUENCY.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["EFG_percentile"]=late_shotclock_shots_top_50_percentile.EFG_PCT.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["avg_percentile"]=late_shotclock_shots_top_50_percentile[['FGA_FREQ_percentile','FGA_percentile','EFG_percentile']].sum(axis=1)
late_shotclock_shots_top_50_percentile.nlargest(n=10,columns="avg_percentile")

,PLAYER_ID,PLAYER_NAME,PLAYER_LAST_TEAM_ID,PLAYER_LAST_TEAM_ABBREVIATION,AGE,GP,G,FGA_FREQUENCY,FGM,FGA,FG_PCT,EFG_PCT,FGA_percentile,FGA_percentile_among_top_50,FGA_FREQ_percentile,EFG_percentile,avg_percentile
24,202704,Reggie Jackson,1610612743,DEN,33.0,19,14,0.135,14,27,0.519,0.630,0.936275,0.871287,0.876238,0.886139,2.698651
28,1630692,Jordan Goodwin,1610612756,PHX,25.0,20,12,0.220,11,26,0.423,0.538,0.914216,0.826733,0.990099,0.752475,2.656790
119,201580,JaVale McGee,1610612758,SAC,35.0,16,8,0.224,10,13,0.769,0.769,0.674020,0.341584,1.000000,0.965347,2.639366
21,1629012,Collin Sexton,1610612762,UTA,24.0,20,14,0.149,14,28,0.500,0.536,0.943627,0.886139,0.920792,0.737624,2.602043
10,202710,Jimmy Butler,1610612748,MIA,34.0,16,16,0.140,14,33,0.424,0.500,0.975490,0.950495,0.896040,0.628713,2.500243
30,200768,Kyle Lowry,1610612748,MIA,37.0,19,12,0.198,10,26,0.385,0.481,0.914216,0.826733,0.970297,0.594059,2.478572
19,1630578,Alperen Sengun,1610612745,HOU,21.0,17,13,0.103,17,28,0.607,0.661,0.943627,0.886139,0.608911,0.920792,2.473330
1,203999,Nikola Jokic,1610612743,DEN,28.0,18,18,0.131,22,47,0.468,0.489,0.997549,0.995050,0.851485,0.618812,2.467846
120,1629614,Andrew Nembhard,1610612754,IND,23.0,15,7,0.121,9,13,0.692,0.808,0.674020,0.341584,0.806931,0.985149,2.466099
118,1627936,Alex Caruso,1610612741,CHI,29.0,17,9,0.118,10,13,0.769,1.000,0.674020,0.341584,0.787129,1.000000,2.461148


# The "David vs Goliath" Award (presented by Dwyane Wade)*

most shots blocked where the blocker is at least 5 inches shorter than the blockee

In [17]:
blocks=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER1_ID',right_on='PLAYER_ID').merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER3_ID',right_on='PLAYER_ID')
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
111,Reggie Jackson,6
45,Fred VanVleet,5
124,Talen Horton-Tucker,5
13,CJ McCollum,4
71,Josh Hart,4
83,Kyrie Irving,4
93,Matisse Thybulle,4
117,Scottie Barnes,4
4,Alex Caruso,3
11,Bruce Brown,3


# The "Pick On Someone Your Own Size" Award

most shots blocked where the blocker is at least 5 inches taller than the blockee

In [18]:
blocking_player_heights.query('height_difference <= -5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
141,Victor Wembanyama,40
18,Brook Lopez,34
22,Chet Holmgren,28
126,Rudy Gobert,24
9,Anthony Davis,23
39,Dereck Lively II,21
90,Kristaps Porzingis,19
61,Jakob Poeltl,16
109,Myles Turner,16
77,Joel Embiid,15


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award (presented by Carmelo Anthony)

Lowest contested rebound percentage, minimum 50% of games played

In [19]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
125,Miles McBride,0.000
220,AJ Green,0.000
382,Furkan Korkmaz,0.000
167,Jaden Hardy,0.037
344,Tyus Jones,0.038
76,Theo Maledon,0.043
206,Marcus Sasser,0.053
326,Doug McDermott,0.053
224,Jacob Gilyard,0.056
434,Aaron Holiday,0.071


alternatively: restricting to players > 6 foot 6 inches in height

In [20]:
above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
47,Furkan Korkmaz,0.000
16,Caleb Houstan,0.077
94,Kessler Edwards,0.091
6,Anthony Black,0.111
136,Peyton Watson,0.147
131,Paolo Banchero,0.150
143,Sam Hauser,0.154
90,Kawhi Leonard,0.165
86,Julian Champagnie,0.167
51,Gordon Hayward,0.169


# The "Glass Cleaner" Award (presented by Dennis Rodman, sponsored by Windex)

Highest contested rebound percentage, minimum 50% of games played (https://www.nba.com/stats/players/rebounding?PerMode=Totals&dir=D&sort=REB_CONTEST_PCT)

In [21]:
reb_w_gp_qualify.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
84,Zeke Nnaji,0.650
446,Mitchell Robinson,0.617
203,Ousmane Dieng,0.545
279,Tristan Thompson,0.543
41,Daniel Gafford,0.540
387,Danuel House Jr.,0.538
127,Isaiah Jackson,0.533
159,Eugene Omoruyi,0.529
348,Richaun Holmes,0.528
130,Day'Ron Sharpe,0.525


alternatively: restricting to players < 6 foot 7 inches in height

In [22]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
40,Danuel House Jr.,0.538
62,Eugene Omoruyi,0.529
13,Ben Sheppard,0.500
102,KJ Martin,0.500
104,Kenneth Lofton Jr.,0.429
159,Terance Mann,0.429
4,Aaron Wiggins,0.424
171,Zion Williamson,0.418
97,Josh Okogie,0.411
131,Moses Moody,0.383
